Runs the fg transform tool 

For Methylation Reaction (with no optimization)

In [ ]:
from fgtransform import oxidationtransform

#Dataset
dataset_filepath = 'data/datasets/QM9/qm9.db'


#H-TYPE TRANSFORMATION
targetlabelH_filepath = 'data/autolabel/Hqm9alllabelpert2/labelpert2ver2.csv'

#pert 3---> 1alcsox
#labelH1id = 21
#labelH2id = 53

#pert 3 ---> 2alcsox
#labelH1id = 33
#labelH2id = 62

#pert 3 ---> terminal alkanes to alkenes
labelH1id = 17
labelH2id = 51

# TARGET EMBEDDING VECTOR TYPE
atom_num = 6

n_molecules = [0,10000]

#OUTPUT
output_filepath1 = 'data/fgtransform/model1/oxidation/termalksoxpert3/init.xyz'
output_filepath2 = 'data/fgtransform/model1/oxidation/termalksoxpert3/trans.xyz'

available_properties = ['energy']

oxidation = oxidationtransform.oxidation(dataset_filepath,targetlabelH_filepath,labelH1id,labelH2id,output_filepath1,output_filepath2,n_molecules,atom_num,available_properties)
number_trans = oxidation.noopt()

In [ ]:
print(number_trans)

FGTransform Analysis

#Run extract embeddings on init and trans

In [ ]:
#Run extract embeddings on init and trans
from extractembeddings import allelementextract

init_dataset_filepath = 'data/fgtransform/model1/remove/1methylmethylene/init.db'
init_save_filepath = 'data/fgtransform/model1/remove/1methylmethylene/initO.csv'
trans_dataset_filepath = 'data/fgtransform/model1/remove/1methylmethylene/trans.db'
trans_save_filepath = 'data/fgtransform/model1/remove/1methylmethylene/transO.csv'

model_filepath = 'data/trainedmodels/model1/best_model'
start = 0
end = 543
n_features = 128
n_layers = [5,6]
all_elements = False
element = 8
qm9=False
available_properties = ['energy']
#label_file =  '../../data/labeldataset/%s/label%s%s.csv' %(element,element,number_inputs)

#run extract on init and trans
allelementextract.extract(qm9,init_dataset_filepath,model_filepath,init_save_filepath,start,end,n_features,n_layers,all_elements,element,available_properties=available_properties)
allelementextract.extract(qm9,trans_dataset_filepath,model_filepath,trans_save_filepath,start,end,n_features,n_layers,all_elements,element,available_properties=available_properties)


transform with average diff vector and use pca on both

In [2]:
from tools.utils import utils_fgtransform, utils_dimred, utils_numericalfiletools

n_molecules = 543
n_features = 128
fg_trans_fildir = 'data/fgtransform/model1/remove/H/1methylmethylene/'

#Calculate average diff vector
data_filepath1 = fg_trans_fildir+ 'initO.csv'
data_filepath2 = fg_trans_fildir+ 'transO.csv'
diff_mean = utils_fgtransform.vecdiffmean(data_filepath1,data_filepath2,n_features,n_molecules)


#Perform transformation
vec_filepath = fg_trans_fildir+ 'initOdiff.csv'
save_filepath = fg_trans_fildir+ 'initOtransform.csv'
utils_fgtransform.add_vectomat(vec_filepath,data_filepath1,n_features,save_filepath)

from tools.utils import utils_fgtransform

true_trans_filepath = fg_trans_fildir+ 'transO.csv'
art_trans_filepath = fg_trans_fildir+ 'initOtransform.csv'
#DO NOT SORT! 
utils_fgtransform.nearest_trans(true_trans_filepath,art_trans_filepath,n_molecules,n_features)




data [[-5.6867400e-01 -1.4290228e-01  2.3530421e+00 ...  5.0000000e+00
  -2.0478730e+03            nan]
 [ 6.6311000e-02 -4.8213986e-01  1.2338269e+00 ...  5.0000000e+00
  -2.0466547e+03            nan]
 [-6.4393280e-01 -5.3058720e-01  3.3208878e+00 ...  5.0000000e+00
  -2.0481885e+03            nan]
 ...
 [ 7.8528140e-01 -3.4994290e-01  1.2211128e+00 ...  5.0000000e+00
  -2.0466794e+03            nan]
 [ 3.0778614e-01  1.9874382e-01  1.1842979e+00 ...  5.0000000e+00
  -2.0471011e+03            nan]
 [-6.7195594e-02 -1.7551467e-01  1.3637297e+00 ...  5.0000000e+00
  -2.0472605e+03            nan]]
vec [ 0.03540061  0.00383385  0.02410092  0.02862699  0.06339722  0.0350505
 -0.02754495 -0.03251793 -0.03506667  0.01160755  0.02888372 -0.00870296
 -0.01681547 -0.06517478  0.00591771  0.02243978 -0.12750441 -0.01381361
  0.01090663 -0.01101679 -0.04200589  0.01174011 -0.01394818 -0.01588982
  0.0025506   0.07091887  0.00167815 -0.01658206 -0.11817853  0.03123358
  0.01774401  0.02793661 -0

In [ ]:
from tools.utils import utils_fgtransform, utils_dimred, utils_numericalfiletools
n_molecules = 68
n_features = 128
fg_trans_fildir = 'data/fgtransform/model1/oxidation/1alcsoxpert2/'


#must stack initOtransform with initO call it bothO
init_filepath = fg_trans_fildir+ 'initO.csv'
inittrans_filepath = fg_trans_fildir+ 'initOtransform.csv'
save_filepath = fg_trans_fildir+ 'bothOtransform.csv'
n_data1 = n_molecules
n_data2 = n_molecules
utils_numericalfiletools.vstacktwofiles(init_filepath,inittrans_filepath,save_filepath,n_data1,n_data2,single_dim=False,skip_header1=1,skip_header2=0)

n_molecules = n_molecules*2
#Dimensionality reduction on new file with init and inittransformed
data_filepath = fg_trans_fildir+ 'bothOtransform.csv'
save_filepath = fg_trans_fildir+ 'initOtransform.csv'
n_components = 128
skip_header =  0
scale_data = False
utils_dimred.pca(data_filepath,n_components,n_features,scale_data,save_filepath, n_molecules,skip_header)

data_filepath = fg_trans_fildir+ 'initOtransformpca.csv'
save_filepath = fg_trans_fildir+ 'initOtransformpcavecs.csv'
#Make vector plotting file from the pca (x1,y1,x2,y2) 
#(half the data is init, half is trans after compilation)
utils_fgtransform.scatters_to_vectors(data_filepath,save_filepath)